LU is not included in table 2 or 4 ub Kozarcanin et al. Apply adjustment based on neighbouring country(ies)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
cwd=os.getcwd()
path_raw_ninja_data=os.path.join(cwd, "../../raw_data/ninja_weather/")
path_adjustment_model=os.path.join(cwd, 'ninja_temp_adjustment.csv')

df_adjustment=pd.read_csv(path_adjustment_model, index_col=0)
df_adjustment=df_adjustment[['slope', 'intercept']]
# focus on neighbouring countries
df_adjustment=df_adjustment.loc[['BE', 'DE', 'FR']]


In [3]:
df_adjustment.mean()

slope        0.926183
intercept    1.332787
dtype: float64

In [4]:
# make a dataframe for the 4 countries of interest
df_raw=pd.DataFrame()
for country in ['BE', 'DE', 'FR', 'LU']:
    _df_raw=pd.read_csv(os.path.join(path_raw_ninja_data, 
                    'ninja_weather_country_%s_merra-2_population_weighted.csv' %country),
                    skiprows=2, usecols=[0,2])
    _df_raw['time']=pd.to_datetime(_df_raw['time'])
    _df_raw=_df_raw.set_index('time')
    exec('df_raw_%s= _df_raw' %country )
    
    if len(df_raw)==0:
        df_raw=_df_raw.rename({'temperature':country},axis=1)
    else:
        df_raw=pd.concat([df_raw, _df_raw.rename({'temperature':country},axis=1)], axis=1)

df_raw.describe()

,BE,DE,FR,LU
count,350640.000000,350640.000000,350640.000000,350640.000000
mean,9.783023,8.785579,11.042347,8.882334
std,7.172334,8.154343,7.263457,8.069127
min,-17.119000,-19.807000,-12.970000,-19.315000
25%,4.474000,2.452000,5.531000,2.662000
50%,9.632000,8.568000,10.644000,8.511000
75%,14.983000,14.919000,16.282000,14.726000
max,37.918000,35.479000,36.465000,39.465000


In [5]:
# compute RMSE and Pearson correlation
from sklearn.metrics import mean_squared_error
for country in ['BE', 'DE', 'FR']:
    print(country)
    print('rmse= ', np.sqrt(mean_squared_error(df_raw[country], df_raw['LU'])))
    print('correlation= ', np.corrcoef((df_raw[country], df_raw['LU'])))


BE
rmse=  1.999931737132872
correlation=  [[1.         0.97940148]
 [0.97940148 1.        ]]
DE
rmse=  1.6293076564950713
correlation=  [[1.         0.97995372]
 [0.97995372 1.        ]]
FR
rmse=  2.9090033872766217
correlation=  [[1.         0.97314826]
 [0.97314826 1.        ]]


In [6]:
# recall that the adjustment for France is one of the least accurate in terms of replicating the monthly HDD
# DE is the closest; followed by BE; we therefore take the average from these two for the adjustment params.

alpha=df_adjustment.loc[['BE', 'DE'], 'slope'].mean()
beta=df_adjustment.loc[['BE', 'DE'], 'intercept'].mean()
print(alpha, beta)

df_LU_adjusted=df_raw['LU']*alpha + beta

# rename the series temperature to match with others
df_LU_adjusted=df_LU_adjusted.rename('temperature')

df_LU_adjusted.to_csv('adjusted_ninja_temperature/adjusted_ninja_temp_LU.csv')

df_LU_adjusted.describe()

0.9074640313131312 1.633786751922825


count    350640.000000
mean          9.694185
std           7.322443
min         -15.893881
25%           4.049456
50%           9.357213
75%          14.997102
max          37.446855
Name: temperature, dtype: float64

In [7]:
# find average HDD per month from 2008 to 2007 (as in table 4)

# use eurostat threshold
threshold=15.


# initialise for each country
_df_HDD=pd.DataFrame(columns=np.arange(1,13,1)) # HDD per month
_df_noheating_hours=pd.DataFrame(columns=np.arange(1,13,1)) # number of hours with no heating in the month

for year, _df_year in df_LU_adjusted.groupby(df_LU_adjusted.index.year):

    _monthly_HDD=np.array([])
    _monthly_sum_temp_ninja=np.array([])
    _monthly_noheating_hours=np.array([])

    for month, _df_month in _df_year.groupby(_df_year.index.month):

        _sum=_df_month.sum() # sum temp

        _s_HDH=threshold-_df_month # series of degree-hours

        _HDD=(_s_HDH).clip(lower=0).sum()/24 # convert heating degree hours to heating degree days for the month

        _noheating_hours=(_s_HDH[_s_HDH<0]).count() # number of no heating hours in the month

        _monthly_HDD=np.append(_monthly_HDD, _HDD)
        _monthly_noheating_hours=np.append(_monthly_noheating_hours, _noheating_hours)
        _monthly_sum_temp_ninja=np.append(_monthly_sum_temp_ninja, _sum)

    _row=pd.DataFrame(columns=np.arange(1,13,1), data=_monthly_HDD.reshape(1,12), index=[year])
    _df_HDD=pd.concat([_df_HDD, _row], axis=0)

    _row1=pd.DataFrame(columns=np.arange(1,13,1), data=_monthly_noheating_hours.reshape(1,12), index=[year])
    _df_noheating_hours=pd.concat([_df_noheating_hours, _row1], axis=0)


In [8]:
_df_HDD.loc[np.arange(2008, 2018)].mean()

1     416.721633
2     357.816973
3     290.453577
4     177.763409
5      90.104325
6      33.830045
7      16.256902
8      19.183515
9      63.467416
10    161.129752
11    263.326499
12    378.637371
dtype: float64

In [9]:
_df_noheating_hours.loc[np.arange(2008, 2018)].mean()

1       0.0
2       0.0
3      17.2
4      85.6
5     249.5
6     432.9
7     562.4
8     531.8
9     302.5
10     87.5
11      5.4
12      0.0
dtype: float64

In [10]:
# take summer months to be June, July, August (same as France)